In [9]:
import torch as t 

## Broadcast广播机制

PyTorch和NumPy中的广播法则定义如下。

1. 所有输入数组都与维度`（n_dim）`最大的数组看齐，维度不足的部分在前面加1补齐。比如，两个数组形状分别为(3,4)和(2,3,1)，前一个会自动将形状补齐为(1,3,4)。

2. 两个数组要么在某一个维度的尺寸一致，要么其中一个数组在该维度的尺寸为1，否则不能计算。比如，形状为(1,3,4)和(2,3,1)的两个数组可以计算，而形状为(1,3,4)和(2,3,2)的两个数组不能计算，这个条件是必须同时满足的。

3. 如果输入数组某个维度的尺寸为1，那么计算时该数组会沿此维度复制扩充成目标的形状。比如，形状为`(1,3,4)`和`(2,3,1)`的两个数组会复制扩充成(2,3,4)。



手动实现广播法则，这样更加直观，也更不容易出错。

- `unsqueeze`、`view`或者`tensor[None]`：为数组的某一维度补1，实现法则（1）。
- `expand(*new_size)`或者`expand_as(tensor_b)`：重复数组，实现法则（3）。因为expand操作不会复制数组，所以不会占用额外的空间。
  
注意：虽然repeat可以实现与expand类似的功能，但是expand是在已经存在的Tensor上创建一个新的视图（view），而repeat会将相同数据复制多份，因而占用额外的内存空间。

In [10]:
a = t.ones(3, 2)
b = t.zeros(2, 3, 1)

# 自动广播法则
# 第一步：a是2维的，b是3维的，所以先在较小维度的a前面补1个维度，
#       即：a.unsqueeze(0)，a的形状变成(1，3，2)，b的形状是(2，3，1),
# 第二步：a和b在第一维和第三维形状不一样，其中一个为1，
#       即：可以利用广播法则扩展，两个形状都变成了(2，3，2)

a = a.unsqueeze(0) #第一步维度扩展
print(a.shape)

c = (a+b)#广播法则
print(c.shape) #相当于实现了对 a的广播法则扩展

torch.Size([1, 3, 2])
torch.Size([2, 3, 2])


In [11]:
a = t.randn(3,2)
print(a.shape)
# 手动广播法则，下面两行操作是等效的
# a.view(1, 3, 2).expand(2, 3, 2)+b.expand(2, 3, 2)
print(a.view(1,3,2).expand(2,3,2).shape)
a[None].expand(2, 3, 2) + b.expand(2, 3, 2)

torch.Size([3, 2])
torch.Size([2, 3, 2])


tensor([[[ 2.1685,  0.5739],
         [ 1.4410, -0.1447],
         [ 1.1596,  1.0706]],

        [[ 2.1685,  0.5739],
         [ 1.4410, -0.1447],
         [ 1.1596,  1.0706]]])

因此整体的广播操作流程就是：
- 先用unsqueeze对维度进行扩展，至少扩展到有相同的维度
- 再对扩展之后的tensor使用expand进行维度广播复制

## 索引操作

In [20]:
a = t.arange(6).view(2,3)
b = a.reshape(3,2,1)
c = b.transpose(2,0)
print(a)
print(b.shape)
print(c.shape)

tensor([[0, 1, 2],
        [3, 4, 5]])
torch.Size([3, 2, 1])
torch.Size([1, 2, 3])


tuple操作

In [22]:
a = t.arange(1,13).view(3,4)
print(a)
print(a[1,2])
print(a[1])

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])
tensor(7)
tensor([5, 6, 7, 8])


slicing操作

切片对象的基本语法形式为：start:stop:step，start是起始索引，stop是终止索引，step是步长（step>0）。从start开始，每隔step个元素取一个，直到stop。

In [26]:
a = t.arange(1,13)
print(a)
print(a[1:8:2])

tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12])
tensor([2, 4, 6, 8])


In [35]:
a = t.arange(12).view(2, 3, 2)
print(a)
print(a[:, 0:2, :])
print(a[1:2, :,:].shape)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]]])
tensor([[[0, 1],
         [2, 3]],

        [[6, 7],
         [8, 9]]])
torch.Size([1, 3, 2])


In [36]:
# a是三维数组
a[1:2, ...]  # 等价于a[1:2,:,:]，补两个 :
a[1, ..., 0] # 等价于a[1,:,0], 补 一个 :

tensor([ 6,  8, 10])

None索引

使用None索引，本质上与使用unsqueeze函数是等价的，都能起到扩展维度的作用。但是，在维度较多的情况下，或者需要对多个维度进行扩展再进行矩阵计算时，使用None索引会更加清晰直观。

In [37]:
a = t.rand(2, 3, 4, 5)
# 在最前面加一个维度，下面两种写法等价
print(a.unsqueeze(0).shape)
print(a[None,...].shape)

torch.Size([1, 2, 3, 4, 5])
torch.Size([1, 2, 3, 4, 5])


In [38]:
b = a.unsqueeze(1).unsqueeze(3).unsqueeze(5)
b.shape

torch.Size([2, 1, 3, 1, 4, 1, 5])

In [39]:
# None索引方法，直接在需要增加的维度上填写None即可
a[:,None,:,None,:,None,:].shape

torch.Size([2, 1, 3, 1, 4, 1, 5])